In [1]:
import os,pandas as pd
file_name = './ml-100k/u.data'
all_ratings = pd.read_csv(file_name , delimiter='\t',names = ['UserID' , 'MovieID', 'Rating', 'Datetime'],header=None)
all_ratings['Datetime'] = pd.to_datetime(all_ratings['Datetime'],unit='s')
all_ratings[:6]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
5,298,474,4,1998-01-07 14:20:06


In [2]:
all_ratings['favorable'] = all_ratings['Rating']>3
favorable_ratings = all_ratings[all_ratings['UserID'].isin(range(200))&all_ratings['favorable']]
favorable_ratings[:6]

,UserID,MovieID,Rating,Datetime,favorable
16,122,387,5,1997-11-11 17:47:39,True
20,119,392,4,1998-01-30 16:13:34,True
21,167,486,4,1998-04-16 14:54:12,True
26,38,95,5,1998-04-13 01:14:54,True
28,63,277,4,1997-10-01 23:10:01,True
29,160,234,5,1997-10-14 20:33:05,True


In [3]:
'''
    1.统计每个用户喜欢过的电影
    2.统计每部电影的被喜欢数
'''
favorated_by_users = {user : frozenset(movieid) for user , movieid in favorable_ratings.groupby('UserID')['MovieID']}
#movies_favorable_num = favorable_ratings.groupby('MovieID').favorable.sum()
#%matplotlib inline
movies_favorable_num = favorable_ratings[['MovieID', 'favorable']].groupby('MovieID').count()
movies_favorable_num.sort_values('favorable',ascending=False)[:5]
#movies_favorable_num.sort_values('favorable',ascending=False)[:5].plot(kind = 'Bar')


,favorable
MovieID,
50,100
100,89
258,83
181,79
174,74


In [39]:
frequent_itemsets = {}
min_support = 50
frequent_itemsets[1] = dict((frozenset((movie_id,)) , row['favorable'])
                           for movie_id , row in movies_favorable_num.iterrows() 
                           if row['favorable'] > min_support)
frequent_itemsets

{1: {frozenset({1}): 66,
  frozenset({7}): 67,
  frozenset({9}): 53,
  frozenset({50}): 100,
  frozenset({56}): 67,
  frozenset({64}): 58,
  frozenset({79}): 58,
  frozenset({98}): 70,
  frozenset({100}): 89,
  frozenset({127}): 70,
  frozenset({172}): 59,
  frozenset({174}): 74,
  frozenset({181}): 79,
  frozenset({258}): 83,
  frozenset({286}): 59,
  frozenset({313}): 60}}

In [60]:
from collections import defaultdict
def find_frequent_itemsets(favorated_by_users , k_1_itemsets , min_support):
    counts = defaultdict(int)
    for user , reviews in favorated_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews-itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))#若other为空集，则用当前子集代替
                    counts[current_superset]+=1
    return {itemset : frequecy for itemset , frequecy in counts.items() if frequecy >= min_support}
#fine_frequent_itemsets(favorated_by_users , frequent_itemsets[1] , 50)


In [66]:
for k in range(2, 20):
    current_frequents_itemsets = find_frequent_itemsets(favorated_by_users ,frequent_itemsets[k-1] , 50)
    frequent_itemsets[k] = current_frequents_itemsets
    if len(current_frequents_itemsets)==0:
        break
print(frequent_itemsets[5])

{frozenset({64, 1, 7, 50, 56}): 70, frozenset({1, 7, 79, 50, 56}): 70, frozenset({1, 98, 7, 50, 56}): 80, frozenset({1, 100, 7, 50, 56}): 65, frozenset({1, 7, 172, 50, 56}): 80, frozenset({1, 7, 174, 50, 56}): 90, frozenset({1, 7, 50, 181, 56}): 75, frozenset({64, 1, 7, 79, 50}): 65, frozenset({64, 1, 98, 7, 50}): 75, frozenset({64, 1, 100, 7, 50}): 60, frozenset({64, 1, 7, 172, 50}): 70, frozenset({64, 1, 7, 174, 50}): 80, frozenset({64, 1, 7, 50, 181}): 75, frozenset({1, 98, 7, 79, 50}): 80, frozenset({1, 100, 7, 79, 50}): 60, frozenset({1, 7, 172, 79, 50}): 75, frozenset({1, 7, 174, 79, 50}): 85, frozenset({1, 7, 79, 50, 181}): 80, frozenset({1, 98, 100, 7, 50}): 60, frozenset({1, 98, 7, 172, 50}): 80, frozenset({1, 98, 7, 174, 50}): 95, frozenset({1, 98, 7, 50, 181}): 80, frozenset({1, 100, 7, 172, 50}): 60, frozenset({1, 100, 7, 174, 50}): 70, frozenset({1, 100, 7, 50, 181}): 70, frozenset({1, 7, 172, 174, 50}): 95, frozenset({1, 7, 172, 50, 181}): 80, frozenset({1, 7, 174, 50, 18